# Challenge 설명
  - 나라 이름(country)이 주어지면 음식(Food)을 맞추고, 그 음식의 레시피(recipe)를 알려주는 프로그램 작성하기
  - 다음과 같은 예시로 흐름을 설명할 수 있음.
    - 한국 사람이 좋아하는 음식의 레시피 >> 음식을 맞추고 레시피를 알려줌.
  - chain을 나누어 작성 진행하기

In [1]:
# llama 3.2 모델 시용 선언
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")

In [2]:
from langchain_core.prompts import PromptTemplate

# food_prompt_template: 국가명을 입력받아 해당 국가를 대표하는 하나의 음식 이름을 반환하는 프롬프트 템플릿
food_prompt_template = PromptTemplate(
    template = "Guess the representative food in {country}. Return the one of the food name only.",
    input_variables=["country"]
)

# recipe_prompt_template: 음식 이름을 입력받아 순서가 있는 간단한 레시피를 반환하는 프롬프트 템플릿
recipe_prompt_template = PromptTemplate(
    template = "How to make {food}? Return the simple recipe with step number only.",
    input_variables=["food"]
)

In [3]:
from langchain_core.output_parsers import StrOutputParser

# 텍스트 파싱을 위한 output_parser 선언
output_parser = StrOutputParser()


In [4]:
from langchain_core.runnables import RunnablePassthrough

# food_chain: RunnablePassthrough를 사용하여 'country'를 입력받은 후 음식이름을 문자열로 반환하는 체인
food_chain = {"country" : RunnablePassthrough()} | food_prompt_template | llm | output_parser

# recipe_chain: 'food'를 입력받은 후 레시피를 문자열로 반환하는 체인
recipe_chain = recipe_prompt_template | llm | output_parser

In [5]:
from langchain_core.runnables import RunnablePassthrough

# final_chain
# - food_chain + recipe_chain
# - RunnablePassthrough를 사용하여 'country'를 입력받은 후 food_chain을 invoke하여 나온 결과를 'food'로 저장하여 recipe_chain을 invoke한 결과를 반환하는 체인
final_chain = {"country" : RunnablePassthrough()} | {"food": food_chain} | recipe_chain

In [6]:
recipe_str = final_chain.invoke({"Republic of Korea"})

# final_chain 결과 값을 정리하여 표시 
recipe_str.split("\n")

["Here's a simple recipe for Jjajangmyeon:",
 '',
 '**Ingredients:**',
 '',
 '- 200g mung bean paste (also known as sweet potato paste)',
 '- 100g soy sauce',
 '- 50g Gochujang (Korean chili paste)',
 '- 20g sugar',
 '- 10g sesame oil',
 '- 2 cloves garlic, minced',
 '- 1 tablespoon grated ginger',
 '- 4 cups water',
 '- 200g wheat flour',
 '- Salt to taste',
 '',
 '**Step by Step:**',
 '',
 '1. Whisk together soy sauce, Gochujang, sugar, and sesame oil in a bowl.',
 '2. Add mung bean paste and mix until well combined.',
 '3. Add garlic and ginger, and mix again.',
 '4. Gradually add water while stirring the mixture.',
 '5. Knead the dough for 10 minutes using wheat flour.',
 '6. Divide the dough into 8-10 equal pieces.',
 '7. Roll out each piece into a thin sheet, about 1/16 inch thick.',
 '8. Cut the sheets into 2-inch strips.',
 '9. Cook the Jjajangmyeon in boiling water for 5-7 minutes or until it floats to the surface.',
 '10. Drain and serve hot with your favorite toppings.']

# 총평
  1. 필요한 정보를 식별한 후 PromptTemplate 구성함.
    - 의도한 답변이 반환되는 프롬프트를 찾는데 어려움이 있었음.
  
  2. 해당 답변에서 필요한 단어가 문자열로 반환될 수 있도록 StrOutputParser를 선언하여 사용함.
    - Recipe와 같은 긴 내용의 답변에 대해 가독성을 높일 수 있도록 Pydentic 선언 및 JSON형으로의 반환을 시도했으나 실패함.
  
  3. 필요한 정보별로 Chain을 구성함.
    - Chain 구성에 어려움이 있었음.
    - Runnable을 적용해야하는 곳과 안해도 되는 곳의 구분에 어려움이 있었음.

  4. 두 Chain을 연결하기 위해 final_chain을 구성함.
    - Chain 간 연결에 어려움이 있었음.
    - 테스트 진행 시 답변의 오류가 많거나 잘못된 답변의 빈도가 높아 final_chain 디버깅에 어려움이 있었음.
  
  5. 최종 결과값의 가독성을 높이기 위해 반복되는 문자를 식별한 후 .split()을 사용하여 결과를 출력함.